In [1]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# install packages
!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_fulltext.csv')
left_dataset = left_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
# split data into trainings and test split
left_dataset = left_dataset.train_test_split(test_size=0.2)

# Preprocess

In [6]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [7]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text"]) #define column wich should be used for finetuning

In [8]:
# tokenize dataset
tokenized_left_dataset = left_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1807 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1369 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (704 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (982 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1395 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


In [9]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
# apply concatenation to dataset
lm_dataset = tokenized_left_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling

# add pad token to tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", is_decoder = True)

In [ ]:
#finetune the pretrained model on the dataset
training_args = TrainingArguments(
    output_dir="bert_left_fulltext_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

/content/bert_left_fulltext_torch is already a clone of https://huggingface.co/tobijen/bert_left_fulltext_torch. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.814700,3.686358
2,3.595600,3.583159
3,3.498900,3.532732
4,3.410600,3.506462
5,3.354500,3.492242
6,3.311500,3.487548


Several commits (2) will be pushed upstream.


TrainOutput(global_step=56922, training_loss=3.5162101942097026, metrics={'train_runtime': 8811.4254, 'train_samples_per_second': 51.679, 'train_steps_per_second': 6.46, 'total_flos': 2.99634990939648e+16, 'train_loss': 3.5162101942097026, 'epoch': 6.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 32.71


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Jul27_07-17-02_da54ce3cef98/events.out.tfevents.1690442224.da54ce3cef98.1145.1:   0%|        …

Upload file runs/Jul27_07-17-02_da54ce3cef98/events.out.tfevents.1690451112.da54ce3cef98.1145.2:   0%|        …

To https://huggingface.co/tobijen/bert_left_fulltext_torch
   834728f..4698654  main -> main

   834728f..4698654  main -> main

To https://huggingface.co/tobijen/bert_left_fulltext_torch
   4698654..e0f914a  main -> main

   4698654..e0f914a  main -> main



'https://huggingface.co/tobijen/bert_left_fulltext_torch/commit/469865435cbaf15cb55a62e841b6defa72179a8a'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/models/bert_left_fulltext_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/bert_left_fulltext_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'generated_text': 'Gay marriage is a big part of the american conversation about gay marriage. " i think it\'s important to remember that the court is not going to take away the right to marry, " he said. " it\'s important to remember that the court is not going to take away the right to marry. " the court\'s decision to allow same - sex marriage to take place in the us is a major victory for gay rights advocates, who have argued that the court\'s decision to allow same - sex'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is a very important thing to do, ” said rep. adam schiff ( d - calif. ), who chairs the house intelligence committee. “ i think it ’ s important to be clear that the president is not going to be a threat to the american people. ” the president ’ s comments came after a series of tweets from the white house, including one from rep. adam schiff ( d - calif. ), who said the president was “ not a threat to the'}]
education_1  =>  Public education is
[{'gener

In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is a big part of the american conversation about gay marriage. " i think it\'s important to remember that the court is not going to take away the right to marry, " he said. " it\'s important to remember that the court is not going to take away the right to marry. " the court\'s decision to allow same - sex marriage to take place in the us is a major victory for gay rights advocates, who have argued that the court\'s decision to allow same - sex',
 'social_2': 'Abortion is a very important thing to do, ” said rep. adam schiff ( d - calif. ), who chairs the house intelligence committee. “ i think it ’ s important to be clear that the president is not going to be a threat to the american people. ” the president ’ s comments came after a series of tweets from the white house, including one from rep. adam schiff ( d - calif. ), who said the president was “ not a threat to the',
 'education_1': 'Public education is a big part of the problem. ” the president ’ s rem

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "bert_left_fulltext": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)